# FM与FFM

![](https://camo.githubusercontent.com/344cef4db9275393ef6baeb70b2ba88ed1c6fd0f89a6e82e6368a3a4e8b98506/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f426c61636b5370616365475a592f63646e2f696d672f74665f31352e706e67)
![](https://camo.githubusercontent.com/a34108cccbcff84571f989c2d8564751a8ae951ed6cb4f9bf20f1a56f2512c23/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f426c61636b5370616365475a592f63646e2f696d672f74665f31362e706e67)

In [ ]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import DenseFeatures, Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D

## 0.准备工作

In [ ]:
seq = fc.sequence_categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.sequence_categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_wide_col = fc.embedding_column(seq, dimension=1)
target_wide_col = fc.embedding_column(target, dimension=1)
wide_columns = [seq_wide_col, target_wide_col]

seq_v_col = fc.embedding_column(seq, dimension=8)
target_v_col = fc.embedding_column(target, dimension=8)
deep_columns = [seq_v_col, target_v_col]

features={
  "seq": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [2, 0], [2, 1]],
      values=[1100, 1101, 1103, 1102, 1101, 1100, 1103],
      dense_shape=[3, 3]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),
}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1100, 1101, 1103],
       [1102, 1101,    0],
       [1100, 1103,    0]], dtype=int32)>

## 1.wide部分

In [ ]:
w0 = tf.compat.v1.get_variable('w0', shape=[1], trainable=True, initializer=tf.keras.initializers.zeros())
w0

<tf.Variable 'w0:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [ ]:
wide_input_layer = SequenceFeatures(wide_columns, name='wide_input_layer')
wide_part_dict, wide_len_dict = wide_input_layer(features)
wide_part_dict

{'seq_embedding': <tf.Tensor: shape=(3, 3, 1), dtype=float32, numpy=
 array([[[-1.1060104 ],
         [ 0.03242132],
         [ 0.2437939 ]],
 
        [[ 0.08936337],
         [ 0.03242132],
         [ 0.        ]],
 
        [[-1.1060104 ],
         [ 0.2437939 ],
         [ 0.        ]]], dtype=float32)>,
 'target_embedding': <tf.Tensor: shape=(3, 1, 1), dtype=float32, numpy=
 array([[[-0.95335877]],
 
        [[-0.07035454]],
 
        [[ 1.2739147 ]]], dtype=float32)>}

In [ ]:
wide_emb = tf.concat(list(wide_part_dict.values()), axis=1)
wide_out = tf.reduce_sum(wide_emb, axis=1)
wide_out

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-1.783154  ],
       [ 0.05143017],
       [ 0.41169816]], dtype=float32)>

In [ ]:
wide_out + w0

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-1.783154  ],
       [ 0.05143017],
       [ 0.41169816]], dtype=float32)>

## 2.Deep部分

In [ ]:
sequence_feature_layer = SequenceFeatures(deep_columns, name='sequence_features_input_layer')
seq_emb_dict, seq_len_dict = sequence_feature_layer(features)
seq_emb_dict.keys()

dict_keys(['seq_embedding', 'target_embedding'])

In [ ]:
feature_emb = tf.concat(list(seq_emb_dict.values()), axis=1)
feature_emb

<tf.Tensor: shape=(3, 4, 8), dtype=float32, numpy=
array([[[-0.49116445, -0.35251305,  0.61282074,  0.10146669,
         -0.2509978 ,  0.5082938 , -0.21420632,  0.7027508 ],
        [ 0.04026853, -0.392949  , -0.19187245, -0.43497851,
          0.09481965,  0.21987827, -0.07222807, -0.47726765],
        [-0.6324771 ,  0.16274336,  0.03654434, -0.06902962,
         -0.5372363 , -0.05480513, -0.43927777, -0.09249779],
        [-0.10923516, -0.41276297, -0.21383002,  0.46652472,
          0.38668025, -0.539819  ,  0.07658985,  0.0994366 ]],

       [[ 0.4117159 ,  0.15799785, -0.11382999,  0.35348448,
         -0.04937907, -0.14734195,  0.26430064, -0.17084403],
        [ 0.04026853, -0.392949  , -0.19187245, -0.43497851,
          0.09481965,  0.21987827, -0.07222807, -0.47726765],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [-0.08201026,  0.32138237, -0.11599014,  0.04416144,
         -0.2511286 , -0.

In [ ]:
square_sum = tf.square(tf.reduce_sum(feature_emb, axis=1, keepdims=True))  # (batch_size, 1, embed_dim)
sum_square = tf.reduce_sum(tf.square(feature_emb), axis=1, keepdims=True)  # (batch_size, 1, embed_dim)
order_2nd = 0.5 * tf.reduce_sum(square_sum - sum_square, axis=2)  # (batch_size, 1)
order_2nd

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.52444935],
       [-0.38796413],
       [-0.7582538 ]], dtype=float32)>

In [ ]:
fm_output = w0 + wide_out + order_2nd
fm_output

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-2.3076034 ],
       [-0.33653396],
       [-0.34655565]], dtype=float32)>

## 3.FFM

1. 同一个field内，计算FM的输出；
2. 不同的field交叉，选用不同的latent vector
3. 设置不同的emb去交叉计算即可



